# Interpolations between songs
To mesure robustness of latent space.

In [6]:
import os
from keras.models import load_model

from model.embeddings.embeddings import embeddings_to_rolls
from utils.files_utils import get_emb_path, load_pickle, get_model_paths, get_audios_path

2023-08-18 12:02:53.532976: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 12:02:53.907949: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-18 12:02:53.907981: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-08-18 12:02:54.917445: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [32]:
from model.colab_tension_vae.params import init

b, z = 4, 96
init(b, z)
model_name = 'brmf_4b_beta-96'
emb_path = get_emb_path(model_name)

df_emb = load_pickle(emb_path)

In [4]:
df_emb[df_emb['Title'] == 'bwv0415']

,index,Style,Title,roll_id,roll,Embedding
232,232,Bach,bwv0415,2,<roll.guoroll.GuoRoll object at 0x7f1b6c1af700>,"[-0.45050272, 0.93562704, 2.0702503, -0.349569..."
231,231,Bach,bwv0415,1,<roll.guoroll.GuoRoll object at 0x7f1b3f5e0550>,"[-0.09109626, 1.1337503, -0.07333951, 2.02099,..."
233,233,Bach,bwv0415,3,<roll.guoroll.GuoRoll object at 0x7f1b3f584430>,"[0.35600132, 1.1052758, 0.9871259, -0.6432949,..."
230,230,Bach,bwv0415,0,<roll.guoroll.GuoRoll object at 0x7f1b3f5c7670>,"[-0.05794812, 1.2383715, 0.044437602, 1.993123..."


In [38]:
def interpolate(model, song1, song2, audios_path, alpha=0.5):
    e1, e2 = song1.Embedding, song2.Embedding
    subtitle = f"{song1.Title}_{song2.Title}_"
    
    interpolation = alpha * e1 + (1 - alpha) * e2
    
    return embeddings_to_rolls([e1, interpolation, e2], [subtitle + '0', subtitle + alpha, subtitle + '100'], "", model, False, audios_path + 'song_interpolations/', True)

In [9]:
vae_path = get_model_paths(model_name)[1]
vae = load_model(os.path.abspath(vae_path))

audios_path = get_audios_path(model_name)

2023-08-18 12:09:51.342331: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-08-18 12:09:51.343077: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-08-18 12:09:51.343107: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Helicon): /proc/driver/nvidia/version does not exist
2023-08-18 12:09:51.343724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/urania/Documentos/Tesis/venv/lib/python3.8/site-packages/keras/layers/

In [33]:
bachs = df_emb[df_emb['Style'] == 'Bach'].reset_index(drop=True)
mozarts = df_emb[df_emb['Style'] == 'Mozart'].reset_index(drop=True)
ragtimes = df_emb[df_emb['Style'] == 'ragtime'].reset_index(drop=True)
frescos = df_emb[df_emb['Style'] == 'Frescobaldi'].reset_index(drop=True)

In [39]:
from model.embeddings.embeddings import interpolate


songs = [bachs.iloc[0], mozarts.iloc[0], ragtimes.iloc[0], frescos.iloc[0]]

rolls = {}
for i, s1 in enumerate(songs):
    for s2 in songs[i:]:
        if s1['index'] != s2['index']:
            rolls[f"{s1.Style}_to_{s2.Style}"] = interpolate(vae, s1.Embedding, s2.Embedding, s1.Title, s2.Title, audios_path)
rolls

{'Bach_to_Mozart': [<roll.guoroll.GuoRoll at 0x7f1adf548cd0>,
 'Bach_to_ragtime': [<roll.guoroll.GuoRoll at 0x7f1adfe2dfd0>,
 'Bach_to_Frescobaldi': [<roll.guoroll.GuoRoll at 0x7f1adfe2d8e0>,
 'Mozart_to_ragtime': [<roll.guoroll.GuoRoll at 0x7f1adfe2d520>,
 'Mozart_to_Frescobaldi': [<roll.guoroll.GuoRoll at 0x7f1adf5ad220>,
 'ragtime_to_Frescobaldi': [<roll.guoroll.GuoRoll at 0x7f1ae0a638b0>,
  <roll.guoroll.GuoRoll at 0x7f1adf5b3ca0>]}

In [35]:
ragtimes

,index,Style,Title,roll_id,roll,Embedding
0,3255,ragtime,original,25,<roll.guoroll.GuoRoll object at 0x7f1a281c0b80>,"[0.6090737, 1.8612064, 0.6157619, -0.369207, 0..."
1,2793,ragtime,elite,15,<roll.guoroll.GuoRoll object at 0x7f1a2758f7f0>,"[0.12866259, -0.53642553, 0.59958607, 1.07605,..."
2,2714,ragtime,countryclub,21,<roll.guoroll.GuoRoll object at 0x7f1a26b5d8b0>,"[-0.23225582, -0.23756057, 0.8842415, -0.41430..."
3,3480,ragtime,roseleaf,19,<roll.guoroll.GuoRoll object at 0x7f1a262533a0>,"[-0.6969494, -1.0329671, -0.56372416, 1.447108..."
4,2823,ragtime,entertainer,9,<roll.guoroll.GuoRoll object at 0x7f1a3532c0d0>,"[0.28102282, -0.1178565, -0.29302725, 0.678586..."
...,...,...,...,...,...,...
401,2966,ragtime,harlemrag,4,<roll.guoroll.GuoRoll object at 0x7f1a164fa100>,"[0.005767579, 0.0054889694, -0.04340487, -0.39..."
402,3087,ragtime,lilyqueen,10,<roll.guoroll.GuoRoll object at 0x7f1a146261f0>,"[-1.2947371, 0.7714095, -1.018264, 0.43782687,..."
403,2540,ragtime,breeze,30,<roll.guoroll.GuoRoll object at 0x7f1a16951130>,"[0.12902223, 0.004088793, 0.3047232, -0.114078..."
404,2769,ragtime,crush,40,<roll.guoroll.GuoRoll object at 0x7f1a20eac250>,"[0.40098506, 2.259939, -0.12402016, -0.3816617..."
